# Initial Modeling with Smaller Images
I wanted to try working with the images pre-noise reduction (just grayscale and high contrast) to see if there are additional features of interest beyond the veins themselves that my image processing would have eliminated.

After doing this, we get a baseline accuracy of about 75%, with a sensitivity of 77.2% and a specificity of 74.6%. This gives us an idea of how much room there is to improve using proper image preprocessing.

In [1]:
# imports
import numpy as np
import tensorflow as tf
import os

# load data
Xtrainb = np.load("../data/train/Xtrainnew3.npy")
Ytrainb = np.load("../data/train/Ytrainnew3.npy")
Xtestb = np.load("../data/test/Xtestnew3.npy")
Ytestb = np.load("../data/test/Ytestnew3.npy")

# setup CNN using Inception-v3 (using google JAMA paper for this choice)
base_model = tf.keras.applications.inception_v3.InceptionV3(
    include_top=False,
    weights='imagenet',
    input_shape=Xtrainb.shape[1:]
)
# freeze inceptionv3 model
for layer in base_model.layers:
    layer.trainable = False

# add additional layers to iterate on
inputs = tf.keras.Input(shape=Xtrainb.shape[1:])
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x) #sumarizing feature existence rather than location
outputs = tf.keras.layers.Dense(1)(x)
modelb = tf.keras.Model(inputs, outputs)

from keras.callbacks import EarlyStopping, ReduceLROnPlateau

# add some training callbacks
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=3, verbose=1, mode='auto')
# Reducing the Learning Rate if result is not improving. 
reduce_lr = ReduceLROnPlateau(monitor='val_loss', min_delta=0.0004, patience=2, factor=0.1, min_lr=1e-6, mode='auto',
                              verbose=1)

2023-03-24 15:38:24.454912: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-24 15:38:29.072984: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# train and save
# train
modelb.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.AUC()])
modelb.fit(x=Xtrainb, y=Ytrainb, epochs=60, validation_data=(Xtestb, Ytestb), callbacks = [early_stop, reduce_lr])

Epoch 1/60
111/111 [==============================] - 55s 475ms/step - loss: 1.0022 - auc_1: 0.5744 - val_loss: 0.4291 - val_auc_1: 0.6435 - lr: 0.0010
Epoch 2/60
111/111 [==============================] - 55s 494ms/step - loss: 0.4384 - auc_1: 0.6306 - val_loss: 0.4324 - val_auc_1: 0.6211 - lr: 0.0010
Epoch 3/60
111/111 [==============================] - ETA: 0s - loss: 0.4882 - auc_1: 0.5977
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
111/111 [==============================] - 50s 451ms/step - loss: 0.4882 - auc_1: 0.5977 - val_loss: 0.4556 - val_auc_1: 0.6048 - lr: 0.0010
Epoch 4/60
111/111 [==============================] - 49s 446ms/step - loss: 0.3902 - auc_1: 0.6543 - val_loss: 0.4256 - val_auc_1: 0.5870 - lr: 1.0000e-04
Epoch 5/60
111/111 [==============================] - 49s 443ms/step - loss: 0.3786 - auc_1: 0.6610 - val_loss: 0.4167 - val_auc_1: 0.6233 - lr: 1.0000e-04
Epoch 6/60
111/111 [==============================] - 49s 442ms/step - lo

In [6]:
# save
modelb.save("../data/saved_model")

INFO:tensorflow:Assets written to: ../data/saved_model/assets


INFO:tensorflow:Assets written to: ../data/saved_model/assets


In [7]:
# raw accuracy
y_pred = modelb.predict(Xtestb, batch_size=1)
accuracy = 0
for i in range(len(y_pred)):
    if np.round(y_pred[i]) == np.round(Ytestb[i]):
        accuracy += 1
print(f"accuracy: {1-(accuracy/len(y_pred))}")

# get sensitivity and specificity, 0 is pos in orig data
sens = 0
sens_tot = 0
spec = 0
spec_tot = 0
for i in range(len(y_pred)):
    pred = np.round(y_pred[i])
    tru = np.round(Ytestb[i])
    # sensitivity: number of correct positives out of all positives
    if tru == 0:
        sens_tot += 1
        if tru == pred:
            sens += 1
    # specificity: number of correct negatives out of all negatives
    elif tru != 1:
        spec_tot += 1
        if tru != pred:
            spec += 1
#display both
print(f"Sensitivity: {sens/sens_tot}")
print(f"Specificity: {spec/spec_tot}")



887/887 [==============================] - 21s 23ms/step
accuracy: 0.7497181510710259
Sensitivity: 0.22695035460992907
Specificity: 0.2546916890080429
